# The Knowledge Graph HTML generator (N1904-TF)

## Table of content (ToC)<a class="anchor" id="TOC"></a>
* <a href="#bullet1">1 - Introduction</a>
* <a href="#bullet2">2 - Add x-y positons to the nodes</a>
* <a href="#bullet3">3 - Generate the cytoscape based HTML file</a>
* <a href="#bullet4">4 - Required libraries</a>
* <a href="#bullet5">5 - Notebook version details</a>

# 1 - Introduction <a class="anchor" id="bullet1"></a>
##### [Back to ToC](#TOC)

This notebook imports the JSON file with the N1904 knowledge graph data (describing TF nodes and features) and converts it into a downloadable (semi stand-alone) HTML file.

# 2 - Add x-y positons to the nodes <a class="anchor" id="bullet2"></a>
##### [Back to ToC](#TOC)

In [1]:
import json
from pathlib import Path

# Load the knowledge graph from JSON
with open("n1904_knowledge_graph.json", encoding="utf-8") as f:
    kg = json.load(f)

# Load the node positions from JSON
with open("node_positions.json", encoding="utf-8") as f:
    node_positions = json.load(f)

nodes = []
for nodeId, nodeData in kg["nodes"].items():
    # We'll just keep the nodeData['description'] as is if it exists.
    # If there's no description, we can add a fallback or leave it blank.
    if "description" not in nodeData:
        nodeData["description"] = ""
    # Also, you might want to store a "label" that strips the "feature::"/"otype::" prefix.
    # But that's optional. We'll do it here for clarity:
    labelStr = nodeId.replace("feature::", "").replace("otype::", "")

    # Get the position from the node_positions dictionary
    position = node_positions.get(nodeId, {})
    # If the position is empty, use default values (0, 0)
    x = position.get("x", 0)
    y = position.get("y", 0)

    # Create the node element with data
    nodeElem = {
        "data": {
            "id": nodeId,
            "label": labelStr,
            # Merge all the fields from nodeData into "data"
            **nodeData,
            "x": x,
            "y": y
        }
    }
    nodes.append(nodeElem)

edges = []
for e in kg["edges"]:
    # We also include freqDetail, if present
    edgeData = {
        "source": e["from"],
        "target": e["to"],
        "label": e["relation"],
    }
    # If there's a freqDetail, add it
    if "freqDetail" in e:
        edgeData["freqDetail"] = e["freqDetail"]

    edgeElem = {"data": edgeData}
    edges.append(edgeElem)

elements = {"nodes": nodes, "edges": edges}
elements_json = json.dumps(elements, indent=2)

# Write out to a file or use directly
Path("n1904_elements.json").write_text(elements_json, encoding="utf-8")
print("Updated elements JSON saved to 'n1904_elements.json'")

Updated elements JSON saved to 'n1904_elements.json'


# 3 - Generate the cytoscape based HTML file <a class="anchor" id="bullet3"></a>
##### [Back to ToC](#TOC)

In [2]:
import json
from pathlib import Path

html_template = """<!DOCTYPE html>
<html lang='en'>
<head>
  <meta charset='UTF-8'>
  <title>N1904 Graph</title>
  <!-- Cytoscape Core -->
  <script src='https://unpkg.com/cytoscape@3.26.0/dist/cytoscape.min.js'></script>
  <!-- Context menus plugin -->
  <script src='https://unpkg.com/cytoscape-context-menus/cytoscape-context-menus.js'></script>
  <link rel='stylesheet' href='https://unpkg.com/cytoscape-context-menus/cytoscape-context-menus.css' />
  <style>
    body {
      margin: 0;
      font-family: sans-serif;
    }
    #cy {
      width: 100vw;
      height: 100vh;
      display: block;
    }
    .tooltip {
      position: absolute;
      padding: 5px 5px;
      background: rgba(0,0,0,0.75);
      color: white;
      font-size: 12px;
      border-radius: 4px;
      pointer-events: none;
      z-index: 10;
      display: none;
      white-space: pre-wrap;
    }
    .tooltip table {
      border-collapse: collapse;
      margin-top: 5px;
    }
    .tooltip th, .tooltip td {
      border: 1px solid white;
      padding: 2px 6px;
      font-size: 12px;
    }
    .faded {
      opacity: 0.1;
    }
  </style>
</head>
<body>
  <div id='cy'></div>
  <div id='tooltip' class='tooltip'></div>

  <!-- Embedded JSON data -->
  <script id='graph-data' type='application/json'>
REPLACE_JSON_HERE
  </script>

  <script>
    window.onload = function() {
      // Parse the JSON data from the embedded script.
      const rawJson = document.getElementById('graph-data').textContent;
      const jsonData = JSON.parse(rawJson);
      
      // Combine the nodes and edges arrays into one array for processing.
      let elements = [];
      if (jsonData.nodes) {
        elements = elements.concat(jsonData.nodes);
      }
      if (jsonData.edges) {
        elements = elements.concat(jsonData.edges);
      }
      
      // For each node, if data.x and data.y exist, add a position property.
      elements.forEach(ele => {
        if (ele.data && ele.data.x !== undefined && ele.data.y !== undefined) {
          ele.position = { x: ele.data.x, y: ele.data.y };
        }
      });
      
      // Initialize Cytoscape using the 'preset' layout so nodes use the provided positions.
      const cy = cytoscape({
        container: document.getElementById('cy'),
        elements: elements,
        style: [
          {
            selector: 'node',
            style: {
              'label': function(ele) {
                const fullLabel = ele.data('label') || '';
                return fullLabel.replace(/^feature::/, '').replace(/^otype::/, '');
              },
              'text-valign': 'center',
              'text-halign': 'center',
              'color': '#000',
              'font-size': '12px',
              'width': 50,
              'height': 50,
              'background-color': function(ele) {
                const t = ele.data('type');
                if (t === 'node_type') return '#7AB7FF';
                else if (t === 'node_feature') return '#A3D977';
                else if (t === 'edge_feature') return '#FBD46D';
                return '#CCCCCC';
              }
            }
          },
          {
            selector: 'edge',
            style: {
              'label': function(ele) {
                const fullLabel = ele.data('label') || '';
                return fullLabel.replace(/^feature::/, '').replace(/^otype::/, '');
              },
              'width': 2,
              'line-color': '#999',
              'target-arrow-shape': 'triangle',
              'target-arrow-color': '#999',
              'curve-style': 'bezier',
              'font-size': '10px',
              'text-rotation': 'autorotate'
            }
          },
          {
            selector: '.faded',
            style: {
              'opacity': 0.1
            }
          }
        ],
        layout: {
          name: 'preset'
        }
      });
      
      // Initialize context menus for nodes and edges.
      cy.contextMenus({
        menuItems: [
          {
            id: 'info-node',
            content: 'Show Node Info',
            tooltipText: 'Node details',
            selector: 'node',
            onClickFunction: function(evt) {
              const d = evt.target.data();
              let msg = "NODE\\n";
              msg += "Label: " + d.label + "\\n";
              msg += "Type: " + d.type + "\\n";
              msg += "Description: " + (d.description || "(none)") + "\\n";
              msg += "Datatype: " + (d.datatype || "(none)") + "\\n";
              alert(msg);
            },
            hasTrailingDivider: true
          },
          {
            id: 'info-edge',
            content: 'Show Edge Info',
            tooltipText: 'Edge details',
            selector: 'edge',
            onClickFunction: function(evt) {
              const d = evt.target.data();
              let msg = "EDGE\\n";
              msg += "Label: " + (d.label || "") + "\\n";
              msg += "Relation: " + (d.relation || "") + "\\n";
              if (d.freqDetail) {
                msg += "\\nFREQ DETAIL:\\n" + JSON.stringify(d.freqDetail, null, 2);
              }
              alert(msg);
            },
            hasTrailingDivider: true
          }
        ]
      });
      
      const tooltip = document.getElementById('tooltip');
      
      // Display tooltips on mouseover only if the element is not faded.
      cy.on('mouseover', 'node, edge', function(evt) {
        const ele = evt.target;
        if (ele.hasClass('faded')) {
          tooltip.style.display = 'none';
          return;
        }
        
        const d = ele.data();
        if (ele.isNode()) {
          // Build a table for node tooltip details.
          let tipText = "<table>";
          tipText += "<tr><th>Label</th><td>" + d.label + "</td></tr>";
          tipText += "<tr><th>Type</th><td>" + d.type + "</td></tr>";
          tipText += "<tr><th>Description</th><td>" + (d.description || "(none)") + "</td></tr>";
          tipText += "<tr><th>Datatype</th><td>" + (d.datatype || "(none)") + "</td></tr>";
          tipText += "</table>";
          tooltip.innerHTML = tipText;
        } else {
          // For edges, show a table with frequency information if available.
          if (d.freqDetail && Array.isArray(d.freqDetail.freq)) {
            let tableHtml = "<table><thead><tr><th>Value</th><th>Count</th></tr></thead><tbody>";
            for (let pair of d.freqDetail.freq) {
              tableHtml += `<tr><td>${pair[0]}</td><td>${pair[1]}</td></tr>`;
            }
            if (d.freqDetail.hasOwnProperty('total')) {
              tableHtml += `<tr><td><strong>Stated total</strong></td><td><strong>${d.freqDetail.total}</strong></td></tr>`;
            }
            tableHtml += "</tbody></table>";
            tooltip.innerHTML = tableHtml;
          } else {
            tooltip.textContent = "No frequency info.";
          }
        }
        tooltip.style.display = 'block';
      });
      
      // Hide tooltip when the mouse leaves an element.
      cy.on('mouseout', 'node, edge', function() {
        tooltip.style.display = 'none';
      });
      
      // Move the tooltip with the mouse.
      cy.on('mousemove', function(e) {
        tooltip.style.left = (e.originalEvent.pageX + 10) + 'px';
        tooltip.style.top = (e.originalEvent.pageY + 10) + 'px';
      });
      
      // Dynamic highlighting: tap on a node to fade out unrelated elements.
      cy.on('tap', 'node', function(evt) {
        const node = evt.target;
        cy.elements().removeClass('faded');
        const neighborhood = node.closedNeighborhood();
        cy.elements().difference(neighborhood).addClass('faded');
      });
      
      // Reset view on background tap.
      cy.on('tap', function(evt) {
        if (evt.target === cy) {
          cy.elements().removeClass('faded');
        }
      });
    };
  </script>
</body>
</html>
"""

# Load your JSON data (which should have "nodes" and "edges" keys) from file.
DATA_FILE = "n1904_elements.json"  # Adjust this filename as needed.
with open(DATA_FILE, encoding="utf-8") as f:
    elements_data = json.load(f)

# Dump the JSON data as a pretty-printed string.
elements_json = json.dumps(elements_data, indent=2)

# Insert the JSON data into the HTML template.
final_html = html_template.replace("REPLACE_JSON_HERE", elements_json)

# Write the final HTML to file.
OUTPUT_HTML = "n1904_graph_contextmenu.html"
Path(OUTPUT_HTML).write_text(final_html, encoding="utf-8")
print(f"HTML successfully written to '{OUTPUT_HTML}' -- ready to open in a browser!")


HTML successfully written to 'n1904_graph_contextmenu.html' -- ready to open in a browser!


# 4 - Required libraries <a class="anchor" id="bullet4"></a>
##### [Back to ToC](#TOC)

The scripts in this notebook require Python 3.8+ and the following libraries to be installed in the environment:

``` python
    json
    pathlib
```

You can install any missing library from within Jupyter Notebook using either `pip` or `pip3`.

# 5 - Notebook version details<a class="anchor" id="bullet5"></a>
##### [Back to ToC](#TOC)

<div style="float: left;">
  <table>
    <tr>
      <td><strong>Author</strong></td>
      <td>Tony Jurg</td>
    </tr>
    <tr>
      <td><strong>Version</strong></td>
      <td>1.2</td>
    </tr>
    <tr>
      <td><strong>Date</strong></td>
      <td>29 April 2025</td>
    </tr>
  </table>
</div>